# Setup Antennal Lobe Simulation Libraries

In [ ]:
import networkx as nx
from neurokernel.LPU.InputProcessors.StepInputProcessor import StepInputProcessor
import numpy as np
name = 'DM4_step1_'
work_path = ''
out_path = ''
G_path = work_path+'{}.gexf'.format(name)
visual_components = np.load(work_path+'visual_components_{}.npy'.format(name), allow_pickle=True)
visual_neurons = np.load(work_path+'visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
def large_scale_sim(name, G_path, conc = 1., DM4_b = 2.17 * 1e-2, DM4_d = 2.94, DL5_b = 0.1 * 2.94, exp_name = '', LN_3=False, LN_3_I = 10.):
    G = nx.read_gexf(G_path)
    G = nx.MultiDiGraph(G)
    sim_name = name + '_conc_' + str(conc) + exp_name
    def set_br(G):
        for node in G.nodes():
            if '_OTP' in node:
                if 'DC1' in node:
                    G.nodes(data=True)[node]['br'] = 0.
                elif 'DM4' in node:
                    G.nodes(data=True)[node]['br'] = DM4_b
                    G.nodes(data=True)[node]['dr'] = DM4_d
                else:
                    G.nodes(data=True)[node]['br'] = DL5_b
                    G.nodes(data=True)[node]['dr'] = DM4_d
        return G

    G = set_br(G)
    inputs = [i for i in G.nodes() if '_OTP' in i]
    fi = StepInputProcessor('conc', inputs, conc, start=1., stop=3.)
    print('Concentration Level:', conc)
    print('Inputs:', inputs)
    def simulate(G, t, inputs,
        record_var_list = None,
        sample_interval: int = 100,
    ):
        from neurokernel.LPU.LPU import LPU
        from neurokernel.LPU.InputProcessors.BaseInputProcessor import (
            BaseInputProcessor,
        )
        from neurokernel.LPU.InputProcessors.ArrayInputProcessor import (
            ArrayInputProcessor,
        )
        from neurokernel.LPU.OutputProcessors.OutputRecorder import OutputRecorder
        from neurokernel.LPU.OutputProcessors.FileOutputProcessor import FileOutputProcessor

        dt = t[1] - t[0]
        if isinstance(inputs, BaseInputProcessor):
            fi = [inputs]
        elif isinstance(inputs, (list, tuple, np.ndarray)) and isinstance(
            inputs[0], BaseInputProcessor
        ):
            fi = inputs
        elif isinstance(inputs, dict):
            for data in inputs.values():
                assert "uids" in data
                assert "data" in data
                assert isinstance(data["data"], np.ndarray)
            fi = [ArrayInputProcessor(inputs)]
        else:
            raise ValueError("Input not understood")
            
        if LN_3:
            fi += [StepInputProcessor('I', ['LN_3'], LN_3_I, start=1., stop=3.)]
            print('Adding current to LN3.')

        fo = OutputRecorder([('I', None), ('g', None)], sample_interval=sample_interval)
        fo2 = OutputRecorder([('spike_state', None)], sample_interval=1)
        fo3 = OutputRecorder([('V', None)], sample_interval=1)
        lpu = LPU(
            dt,
            "obj",
            G,
            device=0,
            id="EOS",
            input_processors=fi,
            output_processors=[fo, fo2, fo3],
            debug=False,
            manager=False
        )
        lpu.run(steps=len(t))

        fo.to_file(out_path+'fo_{}'.format(sim_name))
        fo2.to_file(out_path+'fo2_{}'.format(sim_name))
        fo3.to_file(out_path+'fo3_{}'.format(sim_name))
        return fi, fo, fo2, fo3, lpu
    return simulate, G, fi

## Simple Simulation Setup

In [ ]:
import time
import gc
# names = ['DM4_DL5_step4_a','DM4_DL5_step4_b'] # a: keep synapses to DL5; b: keep synapses to DM4
names = ['DM4_nov_general']
simulation_traces = {}
for name in names:
    for DM4_b_coeff in [0.0,0.2,0.4,0.6,0.8,1.0]:
        for DL5_b_coeff in [0.0]:
            time.sleep(1)
            conc = 10.
            G_path = '/mnt/server-home/mehmet/alclustering/{}.gexf'.format(name)
            visual_components = np.load('/mnt/server-home/mehmet/alclustering/visual_components_{}.npy'.format(name), allow_pickle=True)
            visual_neurons = np.load('/mnt/server-home/mehmet/alclustering/visual_neurons_{}.npy'.format(name), allow_pickle=True).item()
            simulate, G, fi = large_scale_sim(name, G_path, conc=conc, DM4_b = DM4_b_coeff * 0.3, DM4_d = 2.94, DL5_b = DL5_b_coeff * 0.3, 
                                              exp_name = '_changing_balanced_default_' + str(DM4_b_coeff) + '_' + str(DL5_b_coeff), LN_3=True, LN_3_I = 20.)
            dt = 1e-5
            dur = 4.
            steps = int((dur+dt/2)//dt)
            t = np.arange(steps)*dt
            fi, fo, fo2, fo3, lpu = simulate(G, t, fi)